# Tα-prep-C: Collapse Checkpoint Preparation

**Purpose**: Create reliable collapse-state checkpoints for Tα2 experiments

**Settings**:
- λ = 0.60 (fixed, collapse regime)
- η = 0.4 (symmetric noise)
- 100 epochs
- Seeds: 0-9
- Save .pth for runs with error > 40%

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_Ta_prep_collapse'
NOTEBOOK_ID = 'Ta-prep-C'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/checkpoints', exist_ok=True)
os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID}')
print(f'Checkpoints will be saved to: {SAVE_DIR}/checkpoints/')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# Experiment parameters
BATCH_SIZE = 256
NUM_WORKERS = 4
LR = 0.1
K = 16

# Collapse checkpoint settings
NOISE_RATE = 0.4
LAMBDA = 0.60          # Fixed λ for collapse
EPOCHS = 100
COLLAPSE_THRESHOLD = 0.40  # Save checkpoint if error >= this

N_SEEDS = 10  # Seeds 0-9

print(f'η = {NOISE_RATE}')
print(f'λ = {LAMBDA} (fixed, collapse regime)')
print(f'Epochs = {EPOCHS}')
print(f'Save checkpoint if error >= {COLLAPSE_THRESHOLD}')
print(f'Seeds: 0-{N_SEEDS-1}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, lam, cached_gv_ref):
    crit = nn.CrossEntropyLoss()
    model.train()
    step = cached_gv_ref['step']
    cached_gv = cached_gv_ref['gv']
    
    for x, _, idx in train_loader:
        x, idx = x.to(device), idx.to(device)
        bn, bc = noisy_t[idx], clean_t[idx]
        
        opt.zero_grad()
        loss_s = crit(model(x), bn)
        loss_s.backward(retain_graph=True)
        gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        if step % K == 0 or cached_gv is None:
            opt.zero_grad()
            loss_v = crit(model(x), bc)
            loss_v.backward()
            cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
        
        gs_n = gs / (gs.norm() + 1e-12)
        gv_n = cached_gv / (cached_gv.norm() + 1e-12)
        
        g_mix = (1 - lam) * gs_n + lam * gv_n
        opt.zero_grad()
        i = 0
        for p in model.parameters():
            n = p.numel()
            p.grad = g_mix[i:i+n].view(p.shape).clone()
            i += n
        opt.step()
        step += 1
    
    if sched is not None:
        sched.step()
    
    cached_gv_ref['step'] = step
    cached_gv_ref['gv'] = cached_gv

In [ ]:
def run_collapse_training(seed, train_loader, test_loader, clean_labels, noisy_labels):
    """Train with fixed λ=0.60 to create collapse state checkpoint"""
    
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    
    set_seed(seed)
    model = get_resnet18().to(device)
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    
    cached_gv_ref = {'step': 0, 'gv': None}
    trajectory = []
    
    print(f'    Training with λ={LAMBDA} for {EPOCHS} epochs...')
    
    for ep in range(EPOCHS):
        train_one_epoch(model, train_loader, opt, sched, clean_t, noisy_t, LAMBDA, cached_gv_ref)
        
        if (ep + 1) % 10 == 0:
            err = 1 - evaluate(model, test_loader)
            trajectory.append({'epoch': ep+1, 'error': err})
            print(f'      Ep {ep+1}: err={err:.4f}')
    
    final_error = 1 - evaluate(model, test_loader)
    is_collapsed = final_error >= COLLAPSE_THRESHOLD
    
    # Save checkpoint if collapsed
    checkpoint_path = None
    if is_collapsed:
        checkpoint_path = f'{SAVE_DIR}/checkpoints/collapse_seed{seed:02d}_err{final_error:.3f}.pth'
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'seed': seed,
            'lambda': LAMBDA,
            'noise_rate': NOISE_RATE,
            'epochs': EPOCHS,
            'final_error': final_error,
            'trajectory': trajectory
        }, checkpoint_path)
        print(f'    💾 Checkpoint saved: {checkpoint_path}')
    
    return {
        'seed': seed,
        'lambda': LAMBDA,
        'final_error': final_error,
        'is_collapsed': is_collapsed,
        'checkpoint_path': checkpoint_path,
        'trajectory': trajectory
    }

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# Warmup
m = get_resnet18().to(device)
for _ in range(10): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt_file = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done_seeds = set()

if os.path.exists(ckpt_file):
    results = json.load(open(ckpt_file))
    done_seeds = {r['seed'] for r in results}
    print(f'Loaded: {len(done_seeds)} done')

for seed in range(N_SEEDS):
    if seed in done_seeds:
        continue
    
    run_num = len(results) + 1
    print(f'\n{"="*60}')
    print(f'[{run_num}/{N_SEEDS}] Seed {seed} | {NOTEBOOK_ID}')
    print(f'{"="*60}')
    
    noisy_labels = inject_label_noise(clean_labels, NOISE_RATE, seed)
    
    t0 = time.time()
    result = run_collapse_training(seed, train_loader, test_loader, clean_labels, noisy_labels)
    dt = time.time() - t0
    
    result['experiment_id'] = f'{NOTEBOOK_ID}-{seed:03d}'
    result['eta'] = NOISE_RATE
    result['time'] = dt
    
    results.append(result)
    done_seeds.add(seed)
    
    # Summary
    status = "💀 COLLAPSED" if result['is_collapsed'] else "⚠️ NOT collapsed"
    print(f'\n  {status} | Error: {result["final_error"]:.2%} | {dt/60:.1f}min')
    
    # Stats
    collapsed_results = [r for r in results if r.get('is_collapsed', False)]
    print(f'  Collapsed: {len(collapsed_results)}/{len(results)}')
    
    # Save checkpoint
    json.dump(results, open(ckpt_file, 'w'), indent=2, default=str)
    
    remaining = N_SEEDS - run_num
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    torch.cuda.empty_cache()

print('\n' + '='*60)
print(f'{NOTEBOOK_ID} COMPLETE')
print('='*60)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Save final results
json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2, default=str)
df = pd.DataFrame([{k: v for k, v in r.items() if k != 'trajectory'} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY')
print('='*60)

n_total = len(df)
n_collapsed = df['is_collapsed'].sum()
print(f'\n📊 Results: {n_collapsed}/{n_total} collapsed ({100*n_collapsed/n_total:.0f}%)')

print(f'\n📈 Error Distribution:')
print(f'   Mean: {df["final_error"].mean():.3f}')
print(f'   Std:  {df["final_error"].std():.3f}')
print(f'   Min:  {df["final_error"].min():.3f}')
print(f'   Max:  {df["final_error"].max():.3f}')

print(f'\n💾 Saved Checkpoints:')
for r in results:
    if r['checkpoint_path']:
        print(f'   Seed {r["seed"]}: {r["final_error"]:.2%}')

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 1. Error distribution
ax = axes[0]
colors = ['red' if c else 'blue' for c in df['is_collapsed']]
ax.bar(df['seed'], df['final_error'], color=colors, alpha=0.7, edgecolor='black')
ax.axhline(COLLAPSE_THRESHOLD, color='orange', linestyle='--', linewidth=2, label=f'Threshold ({COLLAPSE_THRESHOLD})')
ax.set_xlabel('Seed', fontsize=12)
ax.set_ylabel('Final Error', fontsize=12)
ax.set_title(f'Collapse Checkpoint Preparation (λ={LAMBDA}, η={NOISE_RATE})', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Trajectory
ax = axes[1]
for r in results:
    if r['trajectory']:
        eps = [t['epoch'] for t in r['trajectory']]
        errs = [t['error'] for t in r['trajectory']]
        color = 'red' if r['is_collapsed'] else 'blue'
        ax.plot(eps, errs, alpha=0.7, color=color, linewidth=1.5, label=f"seed {r['seed']}")
ax.axhline(COLLAPSE_THRESHOLD, color='orange', linestyle='--', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Test Error', fontsize=12)
ax.set_title('Training Trajectories', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_summary.png', dpi=150)
plt.show()

print(f'\n📈 Figure saved to {SAVE_DIR}/figures/')